In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Model training

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Data/Large-image-dataset/training',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 2246 images belonging to 2 classes.


In [ ]:
print(training_set.classes)
unique, counts = np.unique(training_set.classes, return_counts=True)
print(dict(zip(training_set.class_indices, counts)))

[0 0 0 ... 1 1 1]
{'Legal': 773, 'No ball': 1473}


In [ ]:
print(training_set.samples)

2246


In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.utils import shuffle
import numpy as np

# Initialize empty lists for training data
X_train = []
y_train = []

# Iterate through the training set and collect the data
i=1
for x, y in training_set:
    print(i)
    i+=1
    X_train.extend(x)
    y_train.extend(y)
    if len(X_train) >= training_set.samples:
        break
# Convert the lists into numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

# Reshape the image data to a 2D array (flatten each image)
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)

# Apply SMOTE to the reshaped training data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_reshaped, y_train)

# Reshape the resampled data back to its original 4D shape (64x64 images with 3 color channels)
X_resampled = X_resampled.reshape(-1, 64, 64, 3)

# Shuffle the resampled data
X_resampled, y_resampled = shuffle(X_resampled, y_resampled, random_state=42)

# Now you have X_resampled and y_resampled with balanced classes


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71


In [ ]:
print(X_resampled.shape)
print(y_resampled.shape)

(2946, 64, 64, 3)
(2946,)


In [ ]:
new_datagen = ImageDataGenerator(rescale=1./255)

# Create the new training set using the resampled data
training_set_resampled = new_datagen.flow(X_resampled, y_resampled, batch_size=32)
# Convert X_resampled and y_resampled into a TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_resampled, y_resampled))

# Shuffle, batch, and optionally prefetch the data for efficient training
training_set = train_dataset.shuffle(buffer_size=1024).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

# Use this dataset in your model training

In [ ]:
print(y_resampled)
unique, counts = np.unique(y_resampled, return_counts=True)
print(dict(zip(unique, counts)))

[1. 1. 1. ... 0. 1. 1.]
{0.0: 1473, 1.0: 1473}


In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Data/Large-image-dataset/testing',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 262 images belonging to 2 classes.


In [ ]:
cnn = tf.keras.models.Sequential()

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [ ]:
cnn.add(tf.keras.layers.Flatten())

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 10)

Epoch 1/10
92/93 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - accuracy: 0.6045 - loss: 0.6798

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


93/93 ━━━━━━━━━━━━━━━━━━━━ 67s 703ms/step - accuracy: 0.6058 - loss: 0.6782 - val_accuracy: 0.7595 - val_loss: 0.5055
Epoch 2/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 17s 178ms/step - accuracy: 0.7613 - loss: 0.4665 - val_accuracy: 0.8588 - val_loss: 0.3794
Epoch 3/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 16s 174ms/step - accuracy: 0.9022 - loss: 0.2521 - val_accuracy: 0.8702 - val_loss: 0.3592
Epoch 4/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 21s 174ms/step - accuracy: 0.9372 - loss: 0.1609 - val_accuracy: 0.8931 - val_loss: 0.2835
Epoch 5/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 20s 173ms/step - accuracy: 0.9566 - loss: 0.1159 - val_accuracy: 0.8702 - val_loss: 0.4482
Epoch 6/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 20s 167ms/step - accuracy: 0.9689 - loss: 0.0867 - val_accuracy: 0.8740 - val_loss: 0.4841
Epoch 7/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 15s 166ms/step - accuracy: 0.9595 - loss: 0.0991 - val_accuracy: 0.9160 - val_loss: 0.3705
Epoch 8/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 21s 174ms/step - accuracy: 0.9847 - loss: 0.0429 - val_accuracy: 0.904

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the test image
test_image = image.load_img('/content/drive/MyDrive/Data/Large-image-dataset/training/Legal/Legal_0_1068.jpeg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
print(result)

if result[0][0] == 0:
  prediction = 'Legal Ball'
else:
  prediction = 'No Ball'
print(prediction)

cnn.save('/content/drive/MyDrive/final_model.keras')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
[[0.]]
Legal Ball


### Model testing using any image

In [ ]:
cnn.save('/content/drive/MyDrive/final_model.keras')

# Load the model
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/final_model.keras')

# Load the test image
test_image = image.load_img('/content/download.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

# Now you can use the loaded_model for predictions or further training
result = loaded_model.predict(test_image)
print(result)

if result[0][0] == 0:
  prediction = 'Legal Ball'
else:
  prediction = 'No Ball'
print(prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
[[0.]]
Legal Ball
